<center><h1>AI in Web Development</h1></center>

---

<center><h2>Lesson 03</h2></center>


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/snsie/ai-webdev/blob/main/lessons/lesson-03/lesson-03.ipynb)

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
import plotly


In [8]:
iris = datasets.load_iris()

In [30]:
# enc = OneHotEncoder().fit(iris.target)
outputs = LabelBinarizer().fit_transform(iris.target)
# outputs=enc.fit(iris.target)

In [24]:
np.shape(iris.data)

(150, 4)

In [64]:
import numpy as np
from sklearn.model_selection import KFold
X = iris.data
y = iris.target
kf = KFold(n_splits=5,shuffle=True)
kf.get_n_splits(X)
numHiddenNodes=16
print(kf)

for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


KFold(n_splits=5, random_state=None, shuffle=True)


In [54]:
numNodesIn=5
numNodesOut=64

In [55]:
def xavierInit(numNodesIn,numNodesOut):
  wMag= np.sqrt(6.0) /np.sqrt(numNodesIn + numNodesOut);
  return np.random.uniform(-wMag,wMag)
xavierInit(4,64)
# import matplotlib.pyplot as plt



0.0014887662584885941

In [60]:
# weightsToHl=np.matrix(shape=(5,64))
def initializeWeightMatrix(numNodesIn,numNodesOut):
  weightsToHl=np.zeros((numNodesIn,numNodesOut))
  for i in range(numNodesIn-1):
    for j in range(numNodesOut):
      weightsToHl[i,j]=xavierInit(numNodesIn,numNodesOut)
  return weightsToHl


In [67]:
class NnFp():
  """
  computes single forward step of neural network
  """
  def __init__(self,inputs,outputs,numHiddenNodes):
    self.inputs = inputs
    self.outputs = outputs
    self.numHiddenNodes=numHiddenNodes
    self.weightsToHl=np.array([[.15,0.25],[0.2,0.3],[0.35,0.35]])
    self.weightsToOl=np.array([[.4,0.5],[0.45,0.55],[0.6,0.6]])

  def initializeWeights(self,numNodesIn,numNodesOut):
    weightMat=np.zeros((numNodesIn,numNodesOut))
    for i in range(numNodesIn-1):
      for j in range(numNodesOut):
        weightMat[i,j]=xavierInit(numNodesIn,numNodesOut)
    return weightMat

  def sigmoid(self,x,derivative=False):
    output = 1/(1+np.exp(-x))
    if derivative:
      return output*(1-output)
    return output

  def getHlNet(self):
    inputsWithBias = np.array([*self.inputs,1])
    #multiply 1x3 input array with 3x2 weights array
    return inputsWithBias @ self.weightsToHl

  def getHlOutputs(self):
    hlNet = self.getHlNet()
    return self.sigmoid(x=hlNet)

  def getOlNet(self):
    hlOutputs=self.getHlOutputs()
    hlOutputsWithBias=np.array([*hlOutputs,1])
    return hlOutputsWithBias @ self.weightsToOl

  def getOlOutputs(self):
    olNet=self.getOlNet()
    return self.sigmoid(olNet)

network=NnFp(X_train,X_test,numHiddenNodes)
network.inputs.shape

(120, 4)

In [ ]:
# create a new class that inherits the forward pass class and combines 
# the backward pass functions to create a reusable object that computes both 
# the forward and backward passes of the network
class NeuralNet(NnFp):
    """
    updates weights via backpropagation
    """
    def __init__(self,inputs,outputs, lr):
        super().__init__(inputs,outputs)
        # predicted from forward pass outputs
        self.lr=lr
        self.hlOutputs=self.getHlOutputs()
        self.predOutputs = self.getOlOutputs()
    
    # get the change in total error with respect to each ol output
    def get_dE_dOlOut(self):
      predOutputs=self.getOlOutputs()
      targets = self.outputs
      return predOutputs-targets

    def get_dOlOut_dOlNet(self):
      olNet=self.getOlNet()
      return self.sigmoid(olNet,derivative=True)

    def get_dOlNet_dWOl(self):
      hlOutputs=self.getHlOutputs()
      hlOutputsWithBias=np.array([*hlOutputs,1])
      return hlOutputsWithBias.reshape(len(hlOutputsWithBias),1)
      
    def get_dE_dOlNet(self):
      dE_dOlOut=self.get_dE_dOlOut()
      dOlOut_dOlNet=self.get_dOlOut_dOlNet()
      dE_dOlNet=dE_dOlOut*dOlOut_dOlNet
      return dE_dOlNet

    def getPartialWeightsToOl(self):
      dE_dOlNet=self.get_dE_dOlNet()
      dE_dOlNet=dE_dOlNet.reshape(1,len(dE_dOlNet))
      dOlNet_dWOl=self.get_dOlNet_dWOl()
      return dOlNet_dWOl@dE_dOlNet

    def getNextWeightsToOl(self):
      partialWeightsToOl=self.getPartialWeightsToOl()
      nextWeightsToOl=self.weightsToOl-lr*partialWeightsToOl
      return nextWeightsToOl

    ########## updating weights to hidden layer below

    def get_dE_dHlOut(self):
      dE_dOlNet=self.get_dE_dOlNet()
      dE_dOlNet=dE_dOlNet.reshape(np.size(dE_dOlNet),1)
      weightsToOlWithoutBias=np.delete(self.weightsToOl,2,0)
      dOlNet_dHlOut=weightsToOlWithoutBias
      return dOlNet_dHlOut@dE_dOlNet

    def get_dHlOut_dHlNet(self):
      hlNet=self.getHlNet() 
      return self.sigmoid(hlNet,derivative=True)

    def get_dHlNet_dWHl(self):
      inputsWithBias = np.array([*self.inputs,1])
      return inputsWithBias.reshape(len(inputsWithBias),1)

    def get_dE_dHlNet(self):
      dE_dHlOut=self.get_dE_dHlOut().flatten()
      dHlOut_dHlNet=self.get_dHlOut_dHlNet()
      return dE_dHlOut*dHlOut_dHlNet

    # create a function to calculate the change in total error with 
    # respect to each weight connected to the hidden nodes
    def getPartialWeightsToHl(self):
      dE_dHlNet=self.get_dE_dHlNet()
      dE_dHlNet=dE_dHlNet.reshape(1,len(dE_dHlNet))
      dHlNet_dWHl=self.get_dHlNet_dWHl()
      return dHlNet_dWHl@dE_dHlNet

    def getNextWeightsToHl(self):
      partialWeightsToHl=self.getPartialWeightsToHl()
      nextWeightsToHl=self.weightsToHl-self.lr*partialWeightsToHl
      return nextWeightsToHl

    def getNextWeightsToHl(self):
      partialWeightsToHl=self.getPartialWeightsToHl()
      nextWeightsToHl=self.weightsToHl-lr*partialWeightsToHl
      return nextWeightsToHl

net=NeuralNet(inputs,outputs,lr)
net.getNextWeightsToHl()